In [ ]:
from glob import glob
import os

path = "/home/hulabdl/CMRxRecon2025/R1_mat_folder/"

filelist = glob(os.path.join(path, "*.mat"))
filelist = [file for file in filelist if 'mask' not in file.lower()]

scorepath = "/home/hulabdl/PromptMR-plus/scorelist.txt"

scores = []

with open(scorepath, 'r') as f:
    for line in f:
        scores.append(line.strip().split())

res = []

for cond in scores:
    for file in filelist:
        flag = True
        for keyword in cond:
            if keyword.lower() not in file.lower():
                flag = False

        if flag:
            res.append((cond, file))


for line in res:
    print(line)

print(len(res))


In [ ]:
import h5py

for cond, file in res:
    with h5py.File(file, 'r') as f:
        data = f['kus']
        shape = data.shape
        if len(shape) == 4:
            shape = [1] + list(shape)
        print(file, "\t".join(map(str, shape)), sep="\t")


In [ ]:
import os

def get_mat_files(directory):
    mat_files = []
    # 遍历目录及其子目录
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".mat") and "kus" in file:  # 检查文件扩展名
                mat_files.append(os.path.join(root, file))  # 获取完整路径
    return mat_files

# 示例用法
directory = "/home/hulabdl/PromptMR-plus/valres1"
mat_files = get_mat_files(directory)
print(f"Found .mat files:{len(mat_files)}")

In [ ]:
path='/home/hulabdl/CMRxRecon2025/ChallengeData/T1rho/TrainingSet/FullSample/Center001/UIH_30T_umr780/P002/T1rho.mat'

import h5py
from scipy.io import loadmat as loadmat_scipy

def loadmat_group(group):
    """
    Load a group in Matlab v7.3 format .mat file using h5py.
    """
    data = {}
    for k, v in group.items():
        if isinstance(v, h5py.Dataset):
            data[k] = v[()]
        elif isinstance(v, h5py.Group):
            data[k] = loadmat_group(v)
    return data

def loadmat(filename):
    """
    Load Matlab v7.3 format .mat file using h5py.
    """
    try:
        with h5py.File(filename, 'r') as f:
            data = {}
            for k, v in f.items():
                if isinstance(v, h5py.Dataset):
                    data[k] = v[()]
                elif isinstance(v, h5py.Group):
                    data[k] = loadmat_group(v)
    except OSError as e:
        f = loadmat_scipy(filename)
        data = {key: value for key, value in f.items() if not key.startswith('__')}
    return data

data = loadmat(path)
print(data.keys())
print(data['kspace'].shape)

# from matplotlib import pyplot as plt
# import numpy as np

# print(np.sum(data['mask'][0], axis=0)[0]-21) 
# plt.imshow(data['mask'][0])
# plt.show()

In [ ]:
import torch

def loadmat_group(group):
    """
    Load a group in Matlab v7.3 format .mat file using h5py.
    """
    data = {}
    for k, v in group.items():
        if isinstance(v, h5py.Dataset):
            data[k] = v[()]
        elif isinstance(v, h5py.Group):
            data[k] = loadmat_group(v)
    return data

def loadmat(filename):
    """
    Load Matlab v7.3 format .mat file using h5py.
    """
    try:
        with h5py.File(filename, 'r') as f:
            data = {}
            for k, v in f.items():
                if isinstance(v, h5py.Dataset):
                    data[k] = v[()]
                elif isinstance(v, h5py.Group):
                    data[k] = loadmat_group(v)
    except OSError as e:
        f = loadmat_scipy(filename)
        data = {key: value for key, value in f.items() if not key.startswith('__')}
    return data

for mat in mat_files:
    data = loadmat(mat)
    recon = torch.as_tensor(data['img4ranking'])
    is_nan = not torch.isfinite(recon).any()
    if is_nan:
        print(mat)

In [ ]:
# path='/home/hulabdl/PromptMR-plus/blackblood_kus_ktRadial16.mat'
path = "/home/hulabdl/PromptMR-plus/blackblood_kus_ktRadial8.mat"
# path = "/nas-data/datasets/CMRxRecon2025/TaskR1/MultiCoil/BlackBlood/ValidationSet/UnderSample_TaskR1/Center006/Siemens_30T_Prisma/P018/blackblood_kus_ktRadial16.mat"

import h5py
from scipy.io import loadmat as loadmat_scipy
from matplotlib import pyplot as plt




data = loadmat(path)
print(data.keys())
print(data['img4ranking'].shape)

import torch

tensor = torch.as_tensor(data['img4ranking'])

print(tensor)

plt.figure()
plt.imshow(data['img4ranking'][1], cmap="gray")
plt.axis("off")
plt.show()

In [ ]:
path='/home/hulabdl/CMRxRecon2025/ChallengeData/MultiCoil/T1w/TrainingSet/Mask_TaskAll/Center003/UIH_30T_umr880/P004/T1w_mask_ktRadial16.mat'

import h5py
from scipy.io import loadmat as loadmat_scipy

def loadmat_group(group):
    """
    Load a group in Matlab v7.3 format .mat file using h5py.
    """
    data = {}
    for k, v in group.items():
        if isinstance(v, h5py.Dataset):
            data[k] = v[()]
        elif isinstance(v, h5py.Group):
            data[k] = loadmat_group(v)
    return data

def loadmat(filename):
    """
    Load Matlab v7.3 format .mat file using h5py.
    """
    try:
        with h5py.File(filename, 'r') as f:
            data = {}
            for k, v in f.items():
                if isinstance(v, h5py.Dataset):
                    data[k] = v[()]
                elif isinstance(v, h5py.Group):
                    data[k] = loadmat_group(v)
    except OSError as e:
        f = loadmat_scipy(filename)
        data = {key: value for key, value in f.items() if not key.startswith('__')}
    return data

data = loadmat(path)
print(data['mask'].shape)